# Scraping Twitter for Followers of LotusOak2

The objective of this notebook was to scrape the followers of one of the most active anti-vaxx bots on Twitter. However, part way through our data collection process (including the week of tweet scraping), this user was banned from Twitter. Other than the user information on LotusOak2, this scraped dataset was not used in the final version of this project.

# Load Libraries

In [102]:
import pandas as pd
import numpy as np
import json
import datetime
import time
import requests
import os
from IPython.display import clear_output

# User Defined Functions

Twitter API v2 offers more information in the search, whereas the libraries retrieving followers data with Twitter API v1.1 are limited in the information the results provide. Therefore, we wrote our own functions in order to use Twitter API v2.

## Set Twitter Authorization Codes

In [2]:
tokens_to_use = "albert"

In [3]:
if (tokens_to_use == "albert") :
    ACCESS_TOKEN = '1277460825599508480-7j6rqWR0NZBCPRZ71pUVyZTKwuaBwf'
    ACCESS_SECRET = 'rZ1kGLrc18GJOEBncEA4GCKwUEoQMfBpaVn8qjSXp3rxH'
    CONSUMER_KEY = 'Mvmui6zu16TEK6Q1NnJp3mWOg'
    CONSUMER_SECRET = 'UsK7AvhtSp0oBEV46Pm4sNzRYc3MAidhauTIX1K5tJKYRllgrc'
    #bearer token
    token = 'AAAAAAAAAAAAAAAAAAAAAA2AJwEAAAAAgBoQOaUAxzTzGKEE9nOMssJ01o4%3DZVpQV4BIPqPuLFlUsmjnkxOi8kv0IOgpOa8NgytniNFW0K36q3'
elif (tokens_to_use == "deborah") :
    ACCESS_TOKEN = '168323323-6hftUbBs0WYhDNmtn8c7X2sH8PvvWdri1ABKecBI'
    ACCESS_SECRET = '4cOcQpqjwKorL0jssomGTUy5nW7d81bVz6TWJp8dwlzEO'
    CONSUMER_KEY = 'WcUcc7EOc5Bw4NuHGI5cGgJZ8'
    CONSUMER_SECRET = 'xrhvRXPhaJZEbtBo9kshs3pgYYnWabPdNO5rqJy5toT7JPDY5S'
    #bearer token
    token = 'AAAAAAAAAAAAAAAAAAAAAIFxJwEAAAAAkL0GXCPub0I%2BMW9v3p1CsNBZMrQ%3DWbaCrcrY6miB8TD7E5L5Xrz6ItfTJoxUy3EPDIksC2of4OVySs'

In [4]:
# define sleep countdown to avoid maxing out Twitter query window
def sleep_countdown(sleep_time) :
    while sleep_time >= 0 :
        m, s = divmod(sleep_time, 60)
        h, m = divmod(m, 60)
        time_left = str(h).zfill(2) + ":" + str(m).zfill(2) + ":" + str(s).zfill(2)
        print(time_left + "\r", end = "")
        time.sleep(1)
        sleep_time -= 1

In [5]:
def create_url_followers (user_id, max_return) :
    """URL query followers of a specific user."""
    user_fields = "user.fields=created_at,description,location,name,username,pinned_tweet_id,protected,verified,public_metrics"
#     expansions = "expansions=author_id,referenced_tweets.id"
    max_results = "max_results=" + str(max_return)
    url = "https://api.twitter.com/2/users/{}/followers?{}&{}".format(
        user_id, user_fields, max_results#, expansions
    )
    return url

def create_headers (bearer_token) :
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint (url, headers) :
    response = requests.request("GET", url, headers = headers)
    if (response.status_code != 200) :
        raise Exception(response.status_code, response.text)
    return response.json()

def main (url) :
    bearer_token = token
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    return (json_response)

In [6]:
def find_followers (user_id, max_return) :
    """Function to find followers fo a specific user"""
    url = create_url_followers(user_id, max_return)
    response = main(url)
    response = pd.json_normalize(response["data"])
    return (response)

In [103]:
def get_followers (id_list, max_return) :
    """Function to format the results from the followers query"""
    count = 0
    l = len(id_list)
    error_id = []
    followers = pd.DataFrame()
    source_id = []
    print("# of users to scrape:", l)
    for i in id_list :
        count += 1
        print("Fetching user #", count, "of", l)
        sleep_countdown(60)
        try :
            result = find_followers(i, max_return)
            followers = pd.concat([followers, result], ignore_index = True)
            print(result.shape[0])
            source_id.extend([i] * result.shape[0])
        except :
            print("Rate limit reached. Sleeping 15 mins.")
            error_id.append(i)
            sleep_countdown(900)
            pass
        clear_output(wait = True)
    followers["source_id"] = source_id
    print("Unable to fetch the following ids:", error_id)
    return (followers)

# Fetch Followers From LotusOak2

In [104]:
#set maximum followers to fetch per user
max_followers = 1000

In [1]:
# start with the user ID of LotusOak2
follower_list = ["424664120"]

In [ ]:
followers1 = get_followers(follower_list, max_followers)

In [107]:
followers1.head()

,location,created_at,protected,username,name,verified,description,id,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,pinned_tweet_id,source_id
0,Minnesota,2020-08-05T00:19:00.000Z,False,djdave2020,🇺🇲⚡⚡Djdave1500⚡⚡🇺🇲,False,All♦️Digital⚡ TRUTH ⚡\nThe Blue Pill Makes Me ...,1290804335484510208,8268,8204,22576,7,NaN,424664120
1,NaN,2018-04-14T20:00:29.000Z,True,PhillipMaldon19,Phillip Maldonado,False,,985246433413562368,6,287,13,0,NaN,424664120
2,Jeddah,2019-11-11T22:53:00.000Z,False,DAlaaiessaaboud,د.علاء عيسى عبود( البروفايل الجديد),False,Cosmetic dentist & Smile designer تجميل الأسنا...,1194025173525110790,31,176,726,0,1328542435958517760,424664120
3,NaN,2020-12-11T18:48:46.000Z,False,JanWeil5,Jan_Marc in ND,False,"Guns,Guitars, Good People (mostly Trumpers) an...",1337469235539238912,69,541,356,0,NaN,424664120
4,Deep South,2018-12-10T14:24:35.000Z,False,BWedehase,flipflopflo,False,MAGA & KAG,1072134990383669248,1055,2968,11844,0,1131671877330579463,424664120


In [108]:
#export followers list
followers1.to_csv("output/followers_ids_1000/followers1.csv", index = False)

In [109]:
#drop all duplicated followers
found_followers = followers1["source_id"].drop_duplicates()

In [110]:
#drop all user where followers were already fetched
follower_list = followers1[~followers1["id"].isin(found_followers)]

#drop all users with private account as user information for private accounts cannot be retrieved
follower_list = follower_list[follower_list["protected"] == False]["id"]

In [ ]:
#get 2nd degree followers of LotusOak2 (followers of the followers of LotusOak2)
followers2 = get_followers(follower_list, max_followers)

In [113]:
#export followers list
followers2.to_csv("output/followers_ids_1000/followers2.csv", index = False)

# Filter Followers Network

We fetched the most recent 1,000 followers of LotusOak2 and the most recent 1,000 followers of his followers. This resulted in a very large list of users and connections. In order for the list to be more manageable, we will filter the list down to to include only:

1) second degree followers are also followers of LotusOak2

2) after the above filter, first degree followers must also have followers in order to be included

In [114]:
# #import previously fetched data in case python notebook timeout
# followers1 = pd.read_csv("output/followers_ids_1000/followers1.csv")
# followers1["id"] = followers1["id"].astype(str)
# followers1["pinned_tweet_id"] = followers1["pinned_tweet_id"].astype(str)
# followers1["source_id"] = followers1["source_id"].astype(str)

# followers2 = pd.read_csv("output/followers_ids _1000/followers2.csv")
# followers2["id"] = followers2["id"].astype(str)
# followers2["pinned_tweet_id"] = followers2["pinned_tweet_id"].astype(str)
# followers2["source_id"] = followers2["source_id"].astype(str)

In [115]:
#drop all duplicated rows (will not drop if the same user follows multiple different users)
found_followers = pd.concat([followers1["source_id"], followers2["source_id"]], ignore_index = True).drop_duplicates()

In [116]:
#filter data to include only nodes who are followers of LotusOak2
#2nd degree followers must also be followers of LotusOak2 in order to be part of the filtered list
followers2_filtered = followers2[followers2["id"].isin(followers1["id"])]
followers1_filtered = followers1[followers1["id"].isin(followers2_filtered["source_id"])]
filtered_followers = pd.concat([followers1_filtered, followers2_filtered], ignore_index = True)

In [117]:
#export filtered followers list
filtered_followers.to_csv("output/followers_ids_1000/filtered_followers.csv", index = False)

# Create Edges & Nodes

In [118]:
#create edges
edges = filtered_followers[["source_id", "id"]]
edges.columns = ["source", "target"]

In [142]:
#create nodes
nodes = filtered_followers[["id", "username", "verified", "created_at", "location", "description"]]
nodes = nodes.drop_duplicates("id")

In [119]:
#export edges and nodes
edges.to_csv("output/followers_ids_1000/edges.csv", index = False)
nodes.to_csv("output/followers_ids_1000/nodes.csv", index = False)